# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.


---

In [1]:
# imports
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown, update_display
from enum import StrEnum
import json
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

import gradio as gr
openai = OpenAI()

In [2]:
system_prompt = """
You are a helpful tutor that explains code. You need to provide an answer structured in markdown without code blocks into the following parts:
- Identify the topic of the question (so the user can look for more info)
- Give an ELI5 explanation of the question.
- Give a step by step explanation of the code.
- Ask the user a follow up question or variation of the question to see if they understand the concept.
- Give the answer to the followup question as a spoiler.

IF the last message is the output of a tool call with an structured markdown you need to return it as it is.
"""

In [3]:
#I'm going to create a tool that will be a LLM as a tool. 
# The tool will actually make a separate lLM call and simply rigorously assess if the answer is valid or not
class Enum_Model(StrEnum):
    GPT = 'gpt-4o-mini'
    LLAMA = 'llama3.2:1b'
    GPT_OSS = 'gpt-oss:20b-cloud'
    HAIKU = 'claude-3-5-sonnet-20240620'



def llm_as_tool(input_msg:str):
    # Generate a system prompt for the LLM to critically analyze if a coding problem's solution is correct
    llm_tool_system_prompt = (
        "You are an expert code reviewer. Your task is to rigorously and critically analyze whether the provided solution "
        "correctly solves the stated coding problem. Carefully consider correctness, completeness, and potential edge cases. "
        "Explain your reasoning with supporting details and point out any flaws, omissions, or improvements. "
        "Provide a clear judgment: is the solution correct? If not, why not? "
        "Output your answer using the following structured markdown format:\n\n"
        "## Analysis\n"
        "- **Correctness:** <your comments>\n"
        "- **Completeness:** <your comments>\n"
        "- **Edge Cases:** <your comments>\n"
        "- **Improvements:** <optional improvement suggestions>\n\n"
        "## Judgment\n"
        "<Clearly state whether the solution is correct, and justify your decision.>"
    )

    ollama = OpenAI(base_url="http://localhost:11434/v1")
    print(f'Calling LLM_Tool with input {input_msg[:10]} ...')
    response = ollama.chat.completions.create(
        model="qwen3-coder:480b-cloud",
        messages=[
            {"role": "system", "content": llm_tool_system_prompt},
            {"role": "user", "content": input_msg},
        ]
    )
    answer = response.choices[0].message.content
    print(f'answer: {answer[:50]}')
    return answer

# There's a particular dictionary structure that's required to describe our function:

check_code_tool_def = {
    "name": "check_code_tool",
    "description": "Checks the code solution provided by the user is correct.",
    "parameters": {
        "type": "object",
        "properties": {
            "input_msg": {
                "type": "string",
                "description": "This is a very concised summary of the question the user asked, the proposed exercise, and the answer the user gave",
            },
        },
        "required": ["input_msg"],
        "additionalProperties": False
    }
}

tools = [
    {"type": "function", "function": check_code_tool_def},
    ]

tools_dict = {
    "check_code_tool": llm_as_tool,
}

def handle_tool_calls(message):
    responses = []
    print(f"This is the message in handle_tool_calls: {message}")
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        func = tools_dict.get(tool_call.function.name, lambda **kwargs: "Unknown tool")
        markdown_analysis = func(**arguments)
        responses.append({
            "role": "tool",
            "content": markdown_analysis,
            "tool_call_id": tool_call.id
        })
        print(f"response for a call is {responses}")
    return responses

def read_text_to_speech(history):
    message = history[-1]['content']
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="onyx",    # Also, try replacing onyx with alloy or coral
      input=message
    )
    return response.content

In [4]:
def chat(history,model):
    # history_dicts = [{"role": h["role"], "content": h["content"]} for h in history]
    # messages = [{"role": "system", "content": system_prompt}] + history_dicts + [{"role": "user", "content": message}]
    #model='GPT'
    print(f"Model selected: {type(model)}")
    if isinstance(model, str):
        try:
            model = Enum_Model[model.upper()]
            print(f"Model selected: {model}")
        except KeyError:
            raise ValueError(f"Unknown model: {model}")
    if model == Enum_Model.LLAMA:
        LLM_ENDPOINT="http://localhost:11434/v1"
        client = OpenAI(base_url=LLM_ENDPOINT)
    elif model == Enum_Model.GPT_OSS:
        LLM_ENDPOINT="http://localhost:11434/v1"
        client = OpenAI(base_url=LLM_ENDPOINT)
    elif model == Enum_Model.GPT:
        client = OpenAI()
    elif model == Enum_Model.HAIKU:
        LLM_ENDPOINT="https://api.anthropic.com/v1/"
        client = OpenAI(base_url=LLM_ENDPOINT, api_key=anthropic_api_key)


    #client = OpenAI()
    
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history

    cumulative_response = ""
    history.append({"role": "assistant", "content": ""})

    response = client.chat.completions.create(
        model=model, 
        messages=messages, 
        tools=tools,
        stream=True
    )
    
    tool_calls = {}
    finish_reason = None
    
    for chunk in response:
        delta = chunk.choices[0].delta
        finish_reason = chunk.choices[0].finish_reason
        
        if hasattr(delta, 'content') and delta.content:
            #cumulative_response += delta.content
            #yield cumulative_response
            history[-1]['content'] += delta.content
            yield history
        
        if hasattr(delta, 'tool_calls') and delta.tool_calls:
            for tool_call_delta in delta.tool_calls:
                idx = tool_call_delta.index
                
                if idx not in tool_calls:
                    tool_calls[idx] = {
                        "id": "",
                        "type": "function",
                        "function": {"name": "", "arguments": ""}
                    }
                
                if tool_call_delta.id:
                    tool_calls[idx]["id"] = tool_call_delta.id
                if tool_call_delta.type:
                    tool_calls[idx]["type"] = tool_call_delta.type
                if hasattr(tool_call_delta, 'function') and tool_call_delta.function:
                    if tool_call_delta.function.name:
                        tool_calls[idx]["function"]["name"] = tool_call_delta.function.name
                    if tool_call_delta.function.arguments:
                        tool_calls[idx]["function"]["arguments"] += tool_call_delta.function.arguments
    
    if finish_reason == "tool_calls":
        from types import SimpleNamespace
        
        tool_call_objects = [
            SimpleNamespace(
                id=tool_calls[idx]["id"],
                type=tool_calls[idx]["type"],
                function=SimpleNamespace(
                    name=tool_calls[idx]["function"]["name"],
                    arguments=tool_calls[idx]["function"]["arguments"]
                )
            )
            for idx in sorted(tool_calls.keys())
        ]
        
        message_obj = SimpleNamespace(tool_calls=tool_call_objects)
        print(message_obj)
        tool_responses = handle_tool_calls(message_obj)
        
        assistant_message = {
            "role": "assistant",
            "content": None,
            "tool_calls": [tool_calls[idx] for idx in sorted(tool_calls.keys())]
        }
        
        messages.append(assistant_message)
        messages.extend(tool_responses)
        #yield cumulative_response

        for tool_response in tool_responses:
            history.append({
                "role": "assistant",
                "content": tool_response["content"]
            })
        
        print('--------------------------------')
        print('history', history)
        print('--------------------------------')

        yield history

        #yield assistant_message
    else:
        return

In [6]:
# Callbacks (along with the chat() function above)

def put_message_in_chatbot(message, history):
        history = [{"role":h["role"], "content":h["content"]} for h in history]

        return "", history + [{"role":"user", "content":message}]

# UI definition

with gr.Blocks() as ui:
    with gr.Row():
        model_dropdown = gr.Dropdown(choices=["GPT", "GPT_OSS", "LLAMA","HAIKU"], value="GPT", label="Model") 
        #image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

# Hooking up events to callbacks

    message.submit(put_message_in_chatbot, 
        inputs=[message, chatbot], 
        outputs=[message, chatbot]
        ).then(
        chat, 
            inputs=[chatbot, model_dropdown], 
            outputs=[chatbot]
    ).then(
        read_text_to_speech,
        inputs=chatbot,
        outputs=audio_output
    )

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


Model selected: <class 'str'>
Model selected: gpt-4o-mini
